# Regresión Lineal: Precio de Casas

En este notebook:
- Generamos datos simulados de precios de casas.
- Entrenamos un modelo con **scikit-learn**.
- Implementamos **regresión lineal desde cero** usando **gradiente descendente**.
- Comparamos ambos enfoques.

In [ ]:
# IMPORTAR
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Estilo
plt.style.use('seaborn-v0_8')
sns.set_palette("viridis")

In [ ]:
# CREAR DATOS FAKE
np.random.seed(42)
n = 1000
data = pd.DataFrame({
    'metros_cuadrados': np.random.uniform(40, 200, n),
    'habitaciones': np.random.randint(1, 6, n),
    'antiguedad': np.random.uniform(0, 40, n)
})

data['precio'] = (
    50000 +
    data['metros_cuadrados'] * 1200 +
    data['habitaciones'] * 8000 -
    data['antiguedad'] * 600 +
    np.random.normal(0, 15000, n)
)
data['precio'] = data['precio'].round(0).astype(int)

print("Primeras filas:")
print(data.head())
print(f"\nEstadísticas:")
print(data.describe().round(1))

In [ ]:
# PREPARAR DATOS
X = data[['metros_cuadrados', 'habitaciones', 'antiguedad']]
y = data['precio']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
# ENTRENAMIENTO CON SCIKIT-LEARN
modelo = LinearRegression()
modelo.fit(X_train, y_train)

print("Modelo entrenado (sklearn).")
print("\nCoeficientes:")
print("-------------")
for var, coef in zip(['metros_cuadrados', 'habitaciones', 'antiguedad'], modelo.coef_):
    print(f" {var}: {coef:,.1f}")
print(f"\nIntercepto: {modelo.intercept_:,.0f}")

In [ ]:
# EVALUAR SCIKIT-LEARN
y_pred = modelo.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("=== MÉTRICAS (sklearn) ===")
print(f"MAE:  ${mae:,.0f}")
print(f"RMSE: ${rmse:,.0f}")
print(f"R²:   {r2:.3f} → {r2*100:.1f}%")

In [ ]:
# GRÁFICO REAL vs PREDICHO (sklearn)
plt.figure(figsize=(7,5))
plt.scatter(y_test, y_pred, alpha=0.6, color='teal')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel("Precio Real")
plt.ylabel("Precio Predicho")
plt.title("Real vs Predicho (sklearn)")
plt.grid(True, alpha=0.3)
plt.show()

---
## Regresión Lineal desde Cero con **Gradiente Descendente**

In [ ]:
# REGRESIÓN LINEAL DESDE CERO
print("="*60)
print("REGRESIÓN LINEAL DESDE CERO CON GRADIENTE DESCENDENTE")
print("="*60)

# Añadimos columna de bias
X_train_bias = np.c_[np.ones(X_train.shape[0]), X_train]
X_test_bias = np.c_[np.ones(X_test.shape[0]), X_test]

# Inicializamos parámetros
np.random.seed(42)
theta = np.random.randn(X_train_bias.shape[1])

# Hiperparámetros
learning_rate = 0.01
n_iterations = 1000
m = len(y_train)
loss_history = []

print(f"Entrenando con {n_iterations} iteraciones, α = {learning_rate}...")

for iteration in range(n_iterations):
    y_pred_train = X_train_bias.dot(theta)
    error = y_pred_train - y_train
    mse = np.mean(error ** 2)
    loss_history.append(mse)
    
    gradients = (2/m) * X_train_bias.T.dot(error)
    theta = theta - learning_rate * gradients

print("Entrenamiento completado!")

# Resultados
print(f"\nParámetros aprendidos (GD):")
print(f"  Intercepto: {theta[0]:,.1f}")
for i, var in enumerate(['metros_cuadrados', 'habitaciones', 'antiguedad']):
    print(f"  {var}: {theta[i+1]:,.1f}")

# Predicción y métricas
y_pred_gd = X_test_bias.dot(theta)
mae_gd = mean_absolute_error(y_test, y_pred_gd)
rmse_gd = np.sqrt(mean_squared_error(y_test, y_pred_gd))
r2_gd = r2_score(y_test, y_pred_gd)

print(f"\nMétricas (Gradiente Descendente):")
print(f"  MAE:  ${mae_gd:,.0f}")
print(f"  RMSE: ${rmse_gd:,.0f}")
print(f"  R²:   {r2_gd:.3f}")

In [ ]:
# COMPARACIÓN VISUAL
plt.figure(figsize=(13,5))

# Curva de pérdida
plt.subplot(1, 2, 1)
plt.plot(loss_history, color='purple')
plt.title("Convergencia de la Pérdida (MSE)")
plt.xlabel("Iteración")
plt.ylabel("MSE")
plt.grid(True, alpha=0.3)

# Real vs Predicho
plt.subplot(1, 2, 2)
plt.scatter(y_test, y_pred, alpha=0.5, color='teal', label='sklearn')
plt.scatter(y_test, y_pred_gd, alpha=0.5, color='coral', label='GD manual')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel("Precio Real")
plt.ylabel("Precio Predicho")
plt.title("Real vs Predicho")
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# PREDICCIÓN NUEVA CASA
nueva_casa = pd.DataFrame({
    'metros_cuadrados': [85],
    'habitaciones': [3],
    'antiguedad': [12]
})

nueva_scaled = scaler.transform(nueva_casa)
nueva_bias = np.c_[np.ones(1), nueva_scaled]

pred_sklearn = modelo.predict(nueva_scaled)[0]
pred_gd = nueva_bias.dot(theta)[0]

print(f"\nCasa de 85 m², 3 hab, 12 años:")
print(f"  sklearn: ${pred_sklearn:,.0f}")
print(f"  GD manual: ${pred_gd:,.0f}")